# GEOPARSING HISTORICAL DOCUMENTS

This notebook is proposed by [L. Moncla](https://ludovicmoncla.github.io/) and [K. McDonough](http://kmcdono.com/) as part of the [GéoDISCO project](https://www.msh-lse.fr/projets/geodisco/).


## Overview

In this tutorial, we'll learn about a few different things.

- How to load data from TEI-XML files into a Python dataframe
- Use Python dataframe for simple data analysis
- Test the [PERDIDO API](http://erig.univ-pau.fr/PERDIDO/api.jsp) for preprocessing French texts (part-of-speech tagging)
- Test the [PERDIDO API](http://erig.univ-pau.fr/PERDIDO/api.jsp) for geoparsing (geotagging + geocoding) Encyclopedie articles
- Display custom geotagging results (PERDIDO TEI-XML) with the [displaCy Named Entity Visualizer](https://spacy.io/usage/visualizers)
- Display geocoding results on a map

## Introduction

Geoparsing (also known as toponym resolution) refers to the process of extracting place names from text and assigning geographic coordinates to them.
This involves two main tasks: geotagging and geocoding.
Geotagging consists to identify spans of text referring to place names while geocoding consists to find unambiguous geographic coordinates.


### The PERDIDO Geoparser API 

The [PERDIDO API](http://erig.univ-pau.fr/PERDIDO/) has been developped for extracting and retrieving displacements from unstructured texts. It has initially been developed for French, Spanish and Italian hiking descriptions.

More recently, as part of the [GéoDISCO project](https://www.msh-lse.fr/projets/geodisco/) we have developed a custom version for historical documents and more specifically for the Encyclopédie.


In this tutorial we'll see how to use the PERDIDO API for preprocessing and geoparsing French texts. 
We will apply geoparsing on the Encyclopedie corpus version released by the [ARTFL project](https://artflsrv03.uchicago.edu/philologic4/encyclopedie1117/navigate/1/5236/) and we'll show the limit of geotagging and geocoding historical documents.


### Getting started


You need to register on the PERDIDO website to get your API key: http://erig.univ-pau.fr/PERDIDO/api.jsp

In [2]:
# import of python libraries

import requests
import lxml.etree as etree
import xml.dom.minidom as xml
import os
from zipfile import ZipFile

import pandas as pd

from spacy.tokens import Span
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy import displacy

#!pip3 install -U display_xml
from display_xml import XML  

#!pip3 install geojson
import geojson
import folium
from IPython.display import display


## 1. Loading the data

Here we assume that we have access to a directory with the corpus of documents. 
In our case, documents are XML-TEI files.

In [3]:
path = './data/' # path of the directory containing the corpus of documents

# select one document for testing
file = 'volume09-3630.tei' # Lyon: https://artflsrv03.uchicago.edu/philologic4/encyclopedie1117/navigate/9/3630/

# get the XML-TEI content of the document
root = etree.parse(path + file, etree.XMLParser(remove_blank_text=True)).getroot()

# print the XML-TEI content
print(xml.parseString(etree.tostring(root)).toprettyxml(indent='  ')) 

<?xml version="1.0" ?>
<TEI.2>
  <teiHeader>
    <fileDesc>
      <titleStmt>
        <title>Encyclopédie, Dictionnaire raisonné des 
                sciences, des arts et des métiers, par une 
                Société de Gens de lettres </title>
        <author>Diderot, Denis and Alembert, Jean d'(eds)</author>
        <respStmt>
          <resp/>
        </respStmt>
      </titleStmt>
      <publicationStmt>
        <distributor>
          <name>ARTFL Project</name>
          <address>
            <addrLine>Department of Romance Languages</addrLine>
            <addrLine>University of Chicago</addrLine>
          </address>
        </distributor>
        
	
        <idno/>
        
      
      </publicationStmt>
      <notesStmt>
        <note/>
      </notesStmt>
      <sourceDesc>
        <bibl>
          <title>Encyclopédie, Dictionnaire raisonné des 
                sciences, des arts et des métiers, par une 
                Société de Gens de lettres. Tome 9.</title>
          <

### 1.1 Extracting metadata and content from XML-TEI

In the following cell, we define a function for parsing and extracting metadata and text content from an XML-TEI file.
In this example, we only extract from the metadata the normclass (classification of the article, e.g. 'Géographie'), the head (head word of the article), and the author of the article. Then, we also extract the textual content as raw text.

In [4]:
def getDataFromEDDATEI(file_path, filename):
    file_id = filename[:-4]
    d = []
    try:
        volume = filename[6:8] 
        number = filename[9:-4] 
        head = ''
        normClass = ''
        author = ''
        txtContent = ''
        root = etree.parse(file_path+filename).getroot()
        div1 = root.find('./text/body/div1')
        if len(div1):
            for elt in div1:
                if elt.tag == 'p':
                    txtContent += ''.join(elt.itertext())
                    txtContent = txtContent.replace('\n', ' ').strip()
                elif elt.tag == 'index':
                    if elt.get('type') == 'normclass':
                        normClass = elt.get('value')
                    if elt.get('type') == 'head':
                        head = elt.get('value')
                    if elt.get('type') == 'author':
                        author = elt.get('value')
        d = [filename, volume, number, head, normClass, author, txtContent]
    except etree.XMLSyntaxError as e:
        pass
        #print(filename + ': ' + str(e))
    return d

This function returns a list object containing the filename, number and volume, the head word, the class, the author, and the textual content.
Let see what is the result of this function for the article about Lyon.

In [5]:
getDataFromEDDATEI(path, file)

['volume09-3630.tei',
 '09',
 '3630',
 'LYON',
 'Géographie',
 'Jaucourt',
 'LYON, (Géogr.) grande, riche, belle, ancienne & celebre ville de France, la plus considérable du royaume après Paris, & la capitale du Lyonnois. Elle se nomme en latin Lugdunum, Lugudunum, Lugdumum Segusianorum, Lugdumum Celtarum, &c. Voyez Lugdunum. Lyon fut fondée l\'an de Rome 712, quarante-un ans avant l\'ere chrétienne, par Lucius Munatius Plancus, qui étoit consul avec AEmilius Lepidus. Il la bâtit sur la Sône, au lieu où cette riviere se jette dans le Rhône, & il la peupla des citoyens romains  qui avoient été chassés de Vienne par les Allobroges. On lit dans Gruter une inscription où il est parlé de l\'établissement de cette colonie ; cependant on n\'honora pas Lyon d\'un nom romain ; elle eut le nom gaulois Lugdun, qu\'avoit la montagne aujourd\'hui Forvieres, sur laquelle cette ville fut fondée. Vibius Sequester prétend que ce mot Lugdun signifioit en langue gauloise, montagne du corbeau. Quoi qu\'il

In order to easily analyse and use these data we will now load these information about all the documents in our directory into a [Python dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html):

In [6]:
data = []
for doc in os.listdir(path):
    if doc[-4:] == '.tei':
        data.append(getDataFromEDDATEI(path, doc))
df = pd.DataFrame(data, columns=['filename', 'volume', 'number', 'head', 'normClass', 'author', 'txtContent'])
df = df.dropna()
df = df.sort_values(['volume', 'number']).reset_index(drop = True)

df.head(10) # show the 10 first rows of the dataframe
#df.tail(10) # show the 10 last rows of the dataframe

,filename,volume,number,head,normClass,author,txtContent
0,volume01-1.tei,01,1,Title Page,unclassified,unsigned,"ENCYCLOPÉDIE, DICTIONNAIRE RAISONNÉ DES SCIENC..."
1,volume01-10.tei,01,10,"A, numismatique ou monétaire",unclassified,Mallet,"A, numismatique ou monétaire, sur le revers de..."
2,volume01-100.tei,01,100,ABATAGE,unclassified,Blondel,"ABATAGE, s. m. On dit dans un chantier & sur u..."
3,volume01-1000.tei,01,1000,AFFÉRENT,Pratique,Toussaint,"AFFÉRENT, adj. terme de pratique, qui n'est us..."
4,volume01-1001.tei,01,1001,AFFERMER,Pratique,Toussaint,"AFFERMER, v. act. terme de Pratique, qui signi..."
5,volume01-1002.tei,01,1002,AFFERMIR la bouche d'un cheval,Manège,Eidous,"AFFERMIR la bouche d'un cheval, v. act. (Manég..."
6,volume01-1003.tei,01,1003,AFFERTEMENT,Marine,unsigned,"AFFERTEMENT, s. m. (Marine.) on se sert de ce ..."
7,volume01-1004.tei,01,1004,AFFERTER,Marine,Bellin,"AFFERTER, v. act. (Marine.) c'est loüer un vai..."
8,volume01-1005.tei,01,1005,AFFERTEUR,Marine,Bellin,"AFFERTEUR, s. m. (Marine.) c'est le nom qu'on ..."
9,volume01-1006.tei,01,1006,AFFEURAGE,Coutumes,unsigned,"AFFEURAGE, s. terme de Coûtumes. Voyez Afforag..."


Now we have access to all the attributs and methods of the dataframe object. For instance, we can easily print the number of rows in our dataframe which correspond to the number of articles in our corpus:

In [7]:
n = df.shape[0]
print('There are ' + str(n) + ' articles in the input directory')

There are 73391 articles in the input directory


### 1.2 First look at the data

Now that the data from the XML-TEI files are loaded into a python dataframe, we can have a look at them.
For instance, we can select articles based on their classification in the Encyclopedie.
If we want all articles in 'geography' we can just do as follows: 

In [8]:
# create the list of class that refers to 'Géographie'
normclassGEO = ['Géographie', 'Géographie moderne',
                 'Géographie ancienne', 'Géographie moderne | Géographie ancienne',
                 'Géographie ancienne | Géographie moderne', 'Géographie sacrée', 'Géographie sainte',
                 'Géographie | Histoire ancienne', 'Géographie historique', 'Géographie | Histoire',
                 'Histoire | Géographie', 'Géographie | Histoire naturelle', 'Géographie | Mythologie',
                 'Géographie ancienne | Mythologie', 'Histoire moderne | Géographie',
                 'Géographie ancienne | Géographie sainte', 'Géographie ancienne | Géographie sacrée',
                 'Géographie sacrée | Géographie ancienne', 'Géographie du moyen âge', 'Géographie des Arabes',
                 'Géographie | Commerce', 'Histoire | Géographie ancienne',
                 'Géographie | Histoire ancienne | Histoire moderne', 'Géographie ancienne | Littérature | Histoire',
                 'Histoire naturelle | Géographie', 'Géographie | Histoire ancienne | Mythologie',
                 'Géographie moderne | Commerce', 'Géographie ancienne | Géographie antique',
                 'Géographie moderne | Histoire', 'Géographie | Histoire monastique',
                 'Géographie ancienne | Géographie moderne | Mythologie', 'Géographie ancienne | Histoire',
                 'Géographie ancienne | Littérature | Mythologie', 'Géographie ancienne | Médailles'
                 ]

# query the dataframe for all articles matching one of the class in our list
df_geo = df.loc[df['normClass'].isin(normclassGEO)]
df_geo.head(10)

,filename,volume,number,head,normClass,author,txtContent
70,volume01-1063.tei,01,1063,AFRIQUE,Géographie,Diderot,"* AFRIQUE, (Géog.) l'une des quatre parties pr..."
71,volume01-1064.tei,01,1064,Afrique,Géographie,Diderot,"* Afrique, (Géog.) Port & Ville de Barbarie au..."
72,volume01-1065.tei,01,1065,Afrique,Géographie moderne,Diderot,"* Afrique, (Géog. mod.) petite ville de France..."
79,volume01-1071.tei,01,1071,AGADES,Géographie,Diderot,"* AGADES, (Géog.) Royaume & Ville de même nom..."
84,volume01-1076.tei,01,1076,AGARÉENS,Géographie | Histoire ancienne,Diderot,"* AGARÉENS, (Géog. Hist. anc.) peuples ainsi n..."
92,volume01-1083.tei,01,1083,AGATE (St),Géographie,Diderot,"* AGATE, (St) Géog. petite ville d'Italie au R..."
96,volume01-1087.tei,01,1087,AGDE,Géographie,Diderot,"* AGDE, (Géog.) ville de France en Languedoc, ..."
114,volume01-1103.tei,01,1103,AGEN,Géographie,Diderot,"* AGEN, (Géog.) ancienne ville de France, capi..."
116,volume01-1105.tei,01,1105,AGENOIS,Géographie,Diderot,"* AGENOIS, adj. pris subst. (Geog.) contrée de..."
131,volume01-1119.tei,01,1119,AGGERHUS,Géographie,Diderot,"*AGGERHUS, (Géog.) gouvernement de Norvege, do..."


In [9]:
print('There are ' + str(df_geo.shape[0]) + ' geography articles')

There are 14314 geography articles


Then, we can also make a query based on the value of the data. For instance, we can query all the articles of a specific author:

In [10]:
val = 'Jaucourt'
n = df_geo.loc[df['author'] == val].shape[0]
print(str(n) + ' were written by '+ val)

8287 were written by Jaucourt


We can also easily show the number of articles per author

In [11]:
df_geo.groupby(['author'])["filename"].count()

author
Bellin                               1
Diderot                           1210
Diderot & Bellin                     1
Diderot3                             4
Dumarsais                            1
Jaucourt                          8287
Jaucourt & Diderot2                  1
Jaucourt & Jaucourt                  2
Jaucourt & Rallier des Ourmes        1
Jaucourt & d'Alembert                1
Jaucourt5                            4
La Condamine                         2
Le Blond                             1
Le Romain                            4
Mallet                               4
Mallet & Diderot                     1
Menuret de Chambaud & Jaucourt       1
Robert de Vaugondy                   2
d'Alembert                          23
d'Alembert & Formey                  1
d'Alembert & d'Holbach               1
d'Argenville                         1
d'Holbach                            4
d'Holbach5                           4
de La Chapelle                       4
unsigned          

In [12]:
df_geo.head(10) 

,filename,volume,number,head,normClass,author,txtContent
70,volume01-1063.tei,01,1063,AFRIQUE,Géographie,Diderot,"* AFRIQUE, (Géog.) l'une des quatre parties pr..."
71,volume01-1064.tei,01,1064,Afrique,Géographie,Diderot,"* Afrique, (Géog.) Port & Ville de Barbarie au..."
72,volume01-1065.tei,01,1065,Afrique,Géographie moderne,Diderot,"* Afrique, (Géog. mod.) petite ville de France..."
79,volume01-1071.tei,01,1071,AGADES,Géographie,Diderot,"* AGADES, (Géog.) Royaume & Ville de même nom..."
84,volume01-1076.tei,01,1076,AGARÉENS,Géographie | Histoire ancienne,Diderot,"* AGARÉENS, (Géog. Hist. anc.) peuples ainsi n..."
92,volume01-1083.tei,01,1083,AGATE (St),Géographie,Diderot,"* AGATE, (St) Géog. petite ville d'Italie au R..."
96,volume01-1087.tei,01,1087,AGDE,Géographie,Diderot,"* AGDE, (Géog.) ville de France en Languedoc, ..."
114,volume01-1103.tei,01,1103,AGEN,Géographie,Diderot,"* AGEN, (Géog.) ancienne ville de France, capi..."
116,volume01-1105.tei,01,1105,AGENOIS,Géographie,Diderot,"* AGENOIS, adj. pris subst. (Geog.) contrée de..."
131,volume01-1119.tei,01,1119,AGGERHUS,Géographie,Diderot,"*AGGERHUS, (Géog.) gouvernement de Norvege, do..."


It is possible to show the value of one of the column of our dataframe for a specific row (i.e., article) based on its name. For instance, if we want to know who wrote the article about Lyon or if we want to see its content:

In [13]:
df.loc[df['head'] == 'LYON'].author.item()

'Jaucourt'

In [14]:
df.loc[df['head'] == 'LYON'].txtContent.item()

'LYON, (Géogr.) grande, riche, belle, ancienne & celebre ville de France, la plus considérable du royaume après Paris, & la capitale du Lyonnois. Elle se nomme en latin Lugdunum, Lugudunum, Lugdumum Segusianorum, Lugdumum Celtarum, &c. Voyez Lugdunum. Lyon fut fondée l\'an de Rome 712, quarante-un ans avant l\'ere chrétienne, par Lucius Munatius Plancus, qui étoit consul avec AEmilius Lepidus. Il la bâtit sur la Sône, au lieu où cette riviere se jette dans le Rhône, & il la peupla des citoyens romains  qui avoient été chassés de Vienne par les Allobroges. On lit dans Gruter une inscription où il est parlé de l\'établissement de cette colonie ; cependant on n\'honora pas Lyon d\'un nom romain ; elle eut le nom gaulois Lugdun, qu\'avoit la montagne aujourd\'hui Forvieres, sur laquelle cette ville fut fondée. Vibius Sequester prétend que ce mot Lugdun signifioit en langue gauloise, montagne du corbeau. Quoi qu\'il en soit, la ville de Lyon est presque aussi souvent nommée  Lugudunum dans 

We can also perform a keyword search over the text content of all articles:

In [15]:
val = 'france'
df_2 = df[df['txtContent'].str.contains(val, case=False)]
print(str(df_2.shape[0]) + ' articles contain the word \''+ val + '\'')

4833 articles contain the word 'france'


Another example with the expression "ville de" will extract all articles that contain the expression 'ville de':

In [16]:
df[df['txtContent'].str.contains("ville de", case=False)].head(10)

,filename,volume,number,head,normClass,author,txtContent
71,volume01-1064.tei,01,1064,Afrique,Géographie,Diderot,"* Afrique, (Géog.) Port & Ville de Barbarie au..."
72,volume01-1065.tei,01,1065,Afrique,Géographie moderne,Diderot,"* Afrique, (Géog. mod.) petite ville de France..."
79,volume01-1071.tei,01,1071,AGADES,Géographie,Diderot,"* AGADES, (Géog.) Royaume & Ville de même nom..."
96,volume01-1087.tei,01,1087,AGDE,Géographie,Diderot,"* AGDE, (Géog.) ville de France en Languedoc, ..."
114,volume01-1103.tei,01,1103,AGEN,Géographie,Diderot,"* AGEN, (Géog.) ancienne ville de France, capi..."
118,volume01-1107.tei,01,1107,AGENS de Change & de Banque,Commerce,unsigned,AGENS de Change & de Banque. s. m. pl. (Comm.)...
180,volume01-1165.tei,01,1165,AGNESTIN,Géographie,Diderot,"* AGNESTIN, (Géog.) ville de Transylvanie sur ..."
204,volume01-1188.tei,01,1188,AGOSTA,Géographie,Diderot,"* AGOSTA, (Géog.) ville de Sicile, & port. Lon..."
222,volume01-1204.tei,01,1204,Agreda,Géographie,Diderot,"* Agreda, (Géog.) ville de l'Amérique méridion..."
226,volume01-1210.tei,01,1210,AGRERE,Géographie,Diderot,* AGRERE (Géog.) petite ville de France dans l...


The same with the words 'océan pacifique' and 'mer pacifique'. Which can be used to study the extent of the Encyclopedie on the pacific area:

In [17]:
df[df['txtContent'].str.contains("océan pacifique|mer pacifique", case=False)]


,filename,volume,number,head,normClass,author,txtContent
1683,volume01-2568.tei,01,2568,ANCUD,Géographie moderne,Diderot,"* ANCUD, (Géog. mod.) l'Archipel d'Ancud ou de..."
3320,volume01-4093.tei,01,4093,ARRACIFES,Géographie,Diderot,"* ARRACIFES, (Géog.) une des îles des Larrons,..."
3473,volume01-4236.tei,01,4236,ARTOMAGAN ou AROMAGA,unclassified,Diderot,"* ARTOMAGAN ou AROMAGA, une île des Larrons, d..."
3844,volume01-4583.tei,01,4583,ATAVILLES,Géographie,Diderot,"* ATAVILLES, s. m. pl. (Géog.) peuples du Péro..."
4277,volume01-4987.tei,01,4987,AVIQUIRINA,Géographie,Diderot,"* AVIQUIRINA, (Géog.) île de l'Amérique septen..."
4514,volume01-5207.tei,01,5207,AYUTLAN,Géographie,Diderot,"* AYUTLAN, (Géog.) riviere de l'Amérique septe..."
13220,volume03-2596.tei,03,2596,COCOS (Isle des),Géographie moderne,unsigned,"COCOS (Isle des), Géog. mod. île de l'Amérique..."
37164,volume09-3890.tei,09,3890,MAGELLANIQUE la terre,Géographie,Jaucourt,"MAGELLANIQUE la terre, (Géog.) C'est ainsi que..."
39066,volume10-1660.tei,10,1660,MER,Géographie,d'Alembert & d'Holbach,"MER, s. f. (Géog.) ce terme signifie ordinaire..."
39104,volume10-1695.tei,10,1695,Mer du Sud,Géographie,unsigned,"Mer du Sud, (Géog.) vaste partie de l'Océan, e..."


Then, the same with a more thematic search for instance about 'esclavage': 

In [18]:
df[df['txtContent'].str.contains("esclavage", case=False)]

,filename,volume,number,head,normClass,author,txtContent
50,volume01-1045.tei,01,1045,Affranchi,Droit romain,Mallet & Toussaint,"Affranchi, adj. pris subst. dans le Droit Roma..."
1908,volume01-2779.tei,01,2779,ANNEAU,Histoire ancienne | Histoire moderne,Mallet,"ANNEAU, s. m. (Hist. anc. & mod.) petit corps ..."
2145,volume01-3.tei,01,3,DISCOURS PRÉLIMINAIRE DES EDITEURS,unclassified,d'Alembert,DISCOURS PRÉLIMINAIRE DES EDITEURS. L'Encyclop...
3125,volume01-3913.tei,01,3913,ARISTOCRATIE,Politique,Mallet & Diderot,"ARISTOCRATIE, s. f. (Politiq.) sorte de gouver..."
3127,volume01-3915.tei,01,3915,ARISTOTELISME,unclassified,Yvon5,"ARISTOTELISME, s. m. Aristote, fils de Nicomac..."
...,...,...,...,...,...,...,...
72402,volume17-2994.tei,17,2994,Zones glaciales,Géographie moderne,Jaucourt,"Zones glaciales, (Géog. mod.) les géographes d..."
72619,volume17-3189.tei,17,3189,INVALIDES,Histoire,Collot,"INVALIDES, (Hist.) addition à cet article. L'h..."
72637,volume17-3204.tei,17,3204,TRANSFUGE,Art militaire,Saint-Lambert5,"TRANSFUGE, s. m. (Art. milit.) La plus grande ..."
72641,volume17-3208.tei,17,3208,"VINGTIEME, imposition",Economie politique,Boulanger,"VINGTIEME, imposition, s. m. (Econ. pol.) dans..."


### 1.3 Preprocessing text content

#### Tokenization and part-of-speech (POS) tagging 

In Natural Language Processing (NLP), the main first steps before processing text content consist in tokenizing sentences and words and assigning to each word its grammatical category (Part-of-Speech). Then, this allows the construction of more complex rules or queries than a simple keyword search.
This preprocessing step is language dependent and thus we have to choose the right tool according to the language of our documents. This is a major difficulty when dealing with historical or ancient texts. For instance, for French it is difficult to find a POS tagger for old French as all well known taggers are trained on contemporary corpora.

> McDonough, K., Moncla, L., & van de Camp, M. (2019). Named entity recognition goes to old regime France: geographic text analysis for early modern French corpora. International Journal of Geographical Information Science, 33, 2498–2522.


The PERDIDO API uses [Treetagger](https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/) for part-of-speech tagging. 

Let's have a first try of the PERDIDO API. We will first test the POS service which takes 3 parameters: the API key of the user, the language, and the text content. This service returns the annotated text in TEI-XML format.

In [19]:
api_key = 'demo' # !! replace by yours
lang = 'French'  # currently only available for French
version = 'Encyclopedie' # default: Standard (the standard version has been developped for the analysis of hiking descriptions)

content = df.loc[df['head'] == 'GANGEA'].txtContent.item()

In [20]:
# set the parameters for the PERDIDO POS tagging service
parameters = {'api_key': api_key, 'lang': lang, 'content': content}
POS_WebService = 'http://erig.univ-pau.fr/PERDIDO/api/pos/txt_xml/'

# run the PERDIDO POS service
r = requests.get(POS_WebService, params=parameters)


Now that the processed is done, we can print the result:

In [21]:
print(r.text) # shows the result of the request

<TEI xmlns="http://www.tei-c.org/ns/1.0"><teiHeader><fileDesc><titleStmt><title><!-- title of the resource --></title></titleStmt><publicationStmt><p><!-- Information about distribution of the resource --></p></publicationStmt><sourceDesc><p><!-- Information about source from which the resource derives --></p></sourceDesc></fileDesc></teiHeader><text><body><p><s><w lemma="null" type="NAM">GANGEA</w><w lemma="," type="PUN">,</w><w lemma="(" type="PUN">(</w><w lemma="null" type="NAM">Géogr</w><w lemma="." type="SENT">.</w><w lemma=")" type="PUN">)</w><w lemma="un" type="NUM">une</w><w lemma="du" type="PRP:det">des</w><w lemma="meilleur" type="ADJ">meilleures</w><w lemma="ville" type="NOM">villes</w><w lemma="de" type="PRP">de</w><w lemma="Perse" type="NAM">Perse</w><w lemma="," type="PUN">,</w><w lemma="dans" type="PRP">dans</w><w lemma="le" type="DET:ART">la</w><w lemma="Géorgie" type="NAM">Géorgie</w><w lemma="," type="PUN">,</w><w lemma="capitale" type="NOM">capitale</w><w lemma="de" 

In [22]:
print(xml.parseString(r.text).toprettyxml()) # shows the result in more a readable way


<?xml version="1.0" ?>
<TEI xmlns="http://www.tei-c.org/ns/1.0">
	<teiHeader>
		<fileDesc>
			<titleStmt>
				<title>
					<!-- title of the resource -->
				</title>
			</titleStmt>
			<publicationStmt>
				<p>
					<!-- Information about distribution of the resource -->
				</p>
			</publicationStmt>
			<sourceDesc>
				<p>
					<!-- Information about source from which the resource derives -->
				</p>
			</sourceDesc>
		</fileDesc>
	</teiHeader>
	<text>
		<body>
			<p>
				<s>
					<w lemma="null" type="NAM">GANGEA</w>
					<w lemma="," type="PUN">,</w>
					<w lemma="(" type="PUN">(</w>
					<w lemma="null" type="NAM">Géogr</w>
					<w lemma="." type="SENT">.</w>
					<w lemma=")" type="PUN">)</w>
					<w lemma="un" type="NUM">une</w>
					<w lemma="du" type="PRP:det">des</w>
					<w lemma="meilleur" type="ADJ">meilleures</w>
					<w lemma="ville" type="NOM">villes</w>
					<w lemma="de" type="PRP">de</w>
					<w lemma="Perse" type="NAM">Perse</w>
					<w lemma="," type="PUN">,</w>

In [23]:
XML(bytes(r.text, 'utf-8'), style='colorful') # shows the same result with syntax color

According to the results we can notice that each word have been tokenised and annotated with the XML element <w> containing the attributes lemma and type (POS tag).

## 2. Geoparsing : geotagging + geocoding

Geoparsing is divided into two main tasks: geotagging (NER) and geocoding.

The geotagging service of the PERDIDO API uses a cascade of finite-state transducers defining specific patterns for NER and identification of geographic information (spatial relations, etc.). 
> Mauro Gaio and Ludovic Moncla (2019). “Geoparsing and geocoding places in a dynamic space context.“ In The Semantics of Dynamic Space in French: Descriptive, experimental and formal studies on motion expression, 66, 353.

For our custom version of the PERDIDO Geoparser, the geocoding task uses a simple gazetteer lookup method. We use the French wikiGazetteer (a gazetteer based on Wikipedia and enriched with Geonames data) generated following this work: https://github.com/alan-turing-institute/lwm_GIR19_resolving_places/tree/master/gazetteer_construction
> Mariona Coll Ardanuy, Katherine McDonough, Amrey Krause, Daniel CS Wilson, Kasra Hosseini, and Daniel van Strien. (2019) “Resolving Places, Past and Present: Toponym Resolution in Historical British Newspapers Using Multiple Resources”. In Proceedings of the 13th Workshop on Geographic Information Retrieval (GIR19).

Geographic text analysis research in the digital humanities has focused on projects analyzing modern English-language corpora. 
In this tutorial we propose to highlight the difficulties of extracting and mapping geographical information from historical French texts.
As we'll see in the following, in addition to the problem of language when it comes to historical documents, the early-modern period lacks temporally appropriate gazetteers.

> McDonough, K., Moncla, L., & van de Camp, M. (2019). Named entity recognition goes to old regime France: geographic text analysis for early modern French corpora. International Journal of Geographical Information Science, 33, 2498–2522.


### 2.1 PERDIDO Geoparser

The PERDIDO Geoparsing service (`http://erig.univ-pau.fr/PERDIDO/api/geoparsing/`) takes 6 new parameters:
1. api_key: API key of the user
2. lang: language of the document (currently only available for French)
3. content: textual content to parse
4. mode: indicates if the query uses exact match on the name (mode: *s*) or if it uses also alternate names (mode: *a*). (default : *s*)
5. records_limit: maximum number of records found in gazetteer for each toponym (default: 1)
6. version: indicates the version of the geoparser (Encyclopedie or Standard). Default: Standard (the standard version has been developped for the analysis of hiking descriptions)

The PERDIDO Geoparser returns XML-TEI. The `<name>` element refers to named entities (proper nouns) and the type attribute indicates its class (place, person, etc.). The `<rs>` element refers to extended named entities (e.g. ville d'Egypte). The `<location>` element indicates that geographic coordinates were found during geocoding.  



As we'll see in the next cell, when we apply the PERDIDO Geoparser to the following example: (Volume 1 article 5236, available online from the [ARTFL project](https://artflsrv03.uchicago.edu/philologic4/encyclopedie1117/navigate/1/5236/))

>AZIRUTH (Géographie.) petite ville d'Egypte, sur la côte occidentale de la mer Rouge ; ce n'est presque plus qu'un village.


Three spatial entities are found during geotagging:
1. Aziruth, 
2. petite ville d'Egypte
3. la côte occidentale de la mer Rouge

while only one entity (*Egypte*) is found during geocoding:

```xml
<name type="place" subtype="edda" id="en.2">
   <w lemma="null" type="NPr" xml:id="w9">Egypte</w>
   <location>
      <geo source="wiki">35.4833 24.1333</geo>
   </location>
</name>
```

In [24]:
# set the parameters for the PERDIDO geoparsing service
content = df.loc[df['head'] == 'AZIRUTH'].txtContent.item()
parameters = {'api_key': api_key, 'lang': lang, 'content': content, 'mode': "s", "records_limit": 1, "version": version}

# request the PERDIDO API
r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geoparsing/', params=parameters)

display(XML(bytes(r.text, 'utf-8'), style='colorful')) # shows the PERDIDO-GEOPARSER XML output



In the next cells, we will use the displacy library from spaCy to display the PERDIDO-NER XML output. For this purpose, we define the function `Perdido2displaCy()` in order to transform the PERDIDO-NER XML into a [spaCy](https://spacy.io/) compatible format.

In [25]:
''' function Perdido2displaCy()
    transforms the PERDIDO-NER XML output into spaCy format (for display purpose) '''
def Perdido2displaCy(contentXML):
    vocab = Vocab()
    words = []
    spaces = []
    root = etree.fromstring(bytes(contentXML, 'utf-8'))
    contentTXT = ""
    for w in root.findall('.//w'):
        contentTXT += w.text + ' '
        words.append(w.text)
        spaces.append(True)
    doc = Doc(vocab, words=words, spaces=spaces)
    ents = [] 
    for child in root.findall('.//rs'):
        if not parent_exists(child, 'rs'):
            if 'startT' in child.attrib:
                start = child.get('startT')
                if 'endT' in child.attrib:
                    stop = child.get('endT')
                    if 'type' in child.attrib:
                        if child.get('type') == 'place':
                            type = 'LOC'
                        elif child.get('type') == 'person':
                            type = 'PERSON'
                        else:
                            type = 'MISC'
                    else:
                        type = 'MISC'
                    ents.append(Span(doc, int(start), int(stop), label=type))
    doc.ents = ents
    return doc 

''' function parent_exists() 
    returns True if one of the ancestor of the element child_node have the name name_node''' 
def parent_exists(child_node, name_node):
    try:
        parent_node = next(child_node.iterancestors())
        if parent_node.tag == name_node:
            if 'startT' in parent_node.attrib:
                return True
        return parent_exists(parent_node, name_node)
    except StopIteration:
        return False

In [26]:
doc = Perdido2displaCy(r.text) # transforms the PERDIDO-GEOPARSER XML

displacy.render(doc, style="ent", jupyter=True) # shows the PERDIDO-GEOPARSER XML output using the displacy library

### 2.2 Mapping place names from one article

If you're interested in geocoding, you probably want to display the result on a map. 
There are two solutions, 
1. you can parse the PERDIDO-GEOPARSER XML and extract each `<location>` element in order to get lat/long coordinate of each entity
2. you can use the PERDIDO-GEOCODING service `http://erig.univ-pau.fr/PERDIDO/api/geocoding/`. It takes the same parameters than the geoparsing service and returns the result as *geojson*.

Let's try solution n°2:

In [27]:
''' function get_bounding_box() returns a list containing the bottom left and the top right 
    points in the sequence '''
def get_bounding_box(points):
    bot_left_x = min(point[1] for point in points)
    bot_left_y = min(point[0] for point in points)
    top_right_x = max(point[1] for point in points)
    top_right_y = max(point[0] for point in points)
    return [(bot_left_x, bot_left_y), (top_right_x, top_right_y)]

''' function display_map() display the map using the folium library '''
def display_map(json_data):
    coords = list(geojson.utils.coords(json_data))
    if len(coords) > 0:
        print(str(len(coords))+" records found in gazetteer:")

        m = folium.Map()
        m.fit_bounds(get_bounding_box(coords), max_zoom=5)
        folium.GeoJson(data, name='Toponyms', tooltip=folium.features.GeoJsonTooltip(fields=['id', 'name', 'source'], localize=True)).add_to(m)

        display(m)
    else:
        print("Sorry, no records found in gazetteer for geocoding!")

In [28]:
r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geocoding/', params=parameters)
data = geojson.loads(r.text)


display_map(data)

1 records found in gazetteer:


Now, let's change the parameters. For geocoding, we will now use alternate names and limit the maximum number of records to 5.

In [29]:

parameters = {'api_key': api_key, 
              'lang': lang, 
              'content': content, 
              'mode': "a", 
              "records_limit": 5, 
              "version": version}

r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geoparsing/', params=parameters)
doc = Perdido2displaCy(r.text)

displacy.render(doc, style="ent", jupyter=True)

r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geocoding/', params=parameters)
data = geojson.loads(r.text)

display_map(data)


6 records found in gazetteer:


Let's try again with a maximum number of record of 1:

In [30]:
parameters = {'api_key': api_key, 
              'lang': lang, 
              'content': content, 
              'mode': "a", 
              "records_limit": 1, 
              "version": version}

r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geoparsing/', params=parameters)
doc = Perdido2displaCy(r.text)

displacy.render(doc, style="ent", jupyter=True)

r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geocoding/', params=parameters)
data = geojson.loads(r.text)

display_map(data)


2 records found in gazetteer:


Sometimes place names can be found in the text but there may be no result for geocoding. This means that none of the entity have records found in gazetteer. This is often the case for historical documents and for periods for which no appropriate gazetteer exists.

In [31]:
content = df.loc[df['head'] == 'AZMER'].txtContent.item()


parameters = {'api_key': api_key, 
              'lang': lang, 
              'content': content, 
              'mode': "a", 
              "records_limit": 1, 
              "version": version}
parameters = {'api_key': 'demo', 'lang': 'French', 'content': content, 'mode': "a", "records_limit": 1}

r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geoparsing/', params=parameters)
doc = Perdido2displaCy(r.text)

displacy.render(doc, style="ent", jupyter=True)


r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geocoding/', params=parameters)
data = geojson.loads(r.text)

display_map(data)


Sorry, no records found in gazetteer for geocoding!


In [32]:

content = df_geo.loc[df_geo['head'] == 'DAUPHINE'].txtContent.item()

parameters = {'api_key': api_key, 
              'lang': lang, 
              'content': content, 
              'mode': "s", 
              "records_limit": 1, 
              "version": version}


r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geoparsing/', params=parameters)
doc = Perdido2displaCy(r.text)

displacy.render(doc, style="ent", jupyter=True)


r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geocoding/', params=parameters)
data = geojson.loads(r.text)

display_map(data)

12 records found in gazetteer:


### 2.3 Mapping several articles

In [36]:
df_pacifique = df_geo[df_geo['txtContent'].str.contains("océan pacifique|mer pacifique", case=False)]
df_pacifique

,filename,volume,number,head,normClass,author,txtContent
1683,volume01-2568.tei,01,2568,ANCUD,Géographie moderne,Diderot,"* ANCUD, (Géog. mod.) l'Archipel d'Ancud ou de..."
3320,volume01-4093.tei,01,4093,ARRACIFES,Géographie,Diderot,"* ARRACIFES, (Géog.) une des îles des Larrons,..."
3844,volume01-4583.tei,01,4583,ATAVILLES,Géographie,Diderot,"* ATAVILLES, s. m. pl. (Géog.) peuples du Péro..."
4277,volume01-4987.tei,01,4987,AVIQUIRINA,Géographie,Diderot,"* AVIQUIRINA, (Géog.) île de l'Amérique septen..."
4514,volume01-5207.tei,01,5207,AYUTLAN,Géographie,Diderot,"* AYUTLAN, (Géog.) riviere de l'Amérique septe..."
13220,volume03-2596.tei,03,2596,COCOS (Isle des),Géographie moderne,unsigned,"COCOS (Isle des), Géog. mod. île de l'Amérique..."
37164,volume09-3890.tei,09,3890,MAGELLANIQUE la terre,Géographie,Jaucourt,"MAGELLANIQUE la terre, (Géog.) C'est ainsi que..."
39066,volume10-1660.tei,10,1660,MER,Géographie,d'Alembert & d'Holbach,"MER, s. f. (Géog.) ce terme signifie ordinaire..."
39104,volume10-1695.tei,10,1695,Mer du Sud,Géographie,unsigned,"Mer du Sud, (Géog.) vaste partie de l'Océan, e..."
41752,volume10-557.tei,10,557,"MARIANES, (les îles)",Géographie,unsigned,"MARIANES, (les îles) autrement les îles DAS VE..."


In [52]:
content = ', '.join(df_pacifique['head'].tolist())
content

'ANCUD, ARRACIFES, ATAVILLES, AVIQUIRINA, AYUTLAN, COCOS (Isle des), MAGELLANIQUE la terre, MER, Mer du Sud, MARIANES, (les îles), OCÉAN, Pacifique, NICOYA, Zacatula, la, Zone torride, Zones tempérées'

In [53]:
parameters = {'api_key': api_key, 
              'lang': lang, 
              'content': content, 
              'mode': "s", 
              "records_limit": 1, 
              "version": version}


r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geoparsing/', params=parameters)
doc = Perdido2displaCy(r.text)

displacy.render(doc, style="ent", jupyter=True)


r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geocoding/', params=parameters)
data = geojson.loads(r.text)

display_map(data)

7 records found in gazetteer:


We change the parameter 'mode' to search also for alternate names:

In [54]:
parameters = {'api_key': api_key, 
              'lang': lang, 
              'content': content, 
              'mode': "a", 
              "records_limit": 1, 
              "version": version}


r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geoparsing/', params=parameters)
doc = Perdido2displaCy(r.text)

displacy.render(doc, style="ent", jupyter=True)


r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geocoding/', params=parameters)
data = geojson.loads(r.text)

display_map(data)

10 records found in gazetteer:


Geocoding headwords of articles containing the word 'esclavage'

In [55]:
df_esclavage = df_geo[df_geo['txtContent'].str.contains("esclavage", case=False)].reset_index(drop = True)
content = ', '.join(df_esclavage['head'].tolist())

parameters = {'api_key': api_key, 
              'lang': lang, 
              'content': content, 
              'mode': "a", 
              "records_limit": 1, 
              "version": version}


r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geoparsing/', params=parameters)
doc = Perdido2displaCy(r.text)

displacy.render(doc, style="ent", jupyter=True)


r = requests.get('http://erig.univ-pau.fr/PERDIDO/api/geocoding/', params=parameters)
data = geojson.loads(r.text)

display_map(data)

13 records found in gazetteer:


## 3. Toponym disambiguation using network analysis

In our work, we use this methodoly for constructing a network based on the citation of "géographie" articles between them.
We proposed to use network analysis measures to establish an approximate location, defined by qualitative relations, for each named toponym in EDDA. Throwing a list of decontextualized toponyms at an external resource like Geonames is risky. We therefore hypothesize that defining meaningful links between places can provide essentialinformation to improve disambiguation (and potentially replace resolution as the end goal). We establish connections between places based on the citation of “headword” toponyms (those that appearas headwords of entries) in other EDDA entries.

>Moncla, L., McDonough, K., Vigier, D., Joliveau, T., & Brenon, A. (2019). Toponym disambiguation in historical documents using network analysis of qualitative relationships. Proceedings of the 3rd ACM SIGSPATIAL International Workshop on Geospatial Humanities, 1–4. Chicago, IL, USA.

This method draws on relations in the corpus of EDDA articles, which improves disambiguation at a later stage with an external resource. We suggest the network as an alternative to geospatial representation, a useful proxy when no historical gazetteer exists for the source material's period. Our first experiments have shown that this approach goes beyond a simple text analysis and is able to find relations between toponyms that are not co-occurring in the same documents. Network relations are also usefully compared with disambiguated toponyms to evaluate geographical coverage, and the ways that geographical discourse is expressed, in historical texts.


<table>
  <tr>
    <td> <img src="img/labels_indegree2.png" width ="500px"> </td>
    <td> <img src="img/nodes_betweenness+class2.png" width ="500px" > </td>
  </tr>
  <tr>
    <td>Node and label size indicate in-degree centrality</td>
    <td>Node size indicates betweenness centrality<br/> 
        colors refer to geographic feature types <br/> 
        (city: red, hydronym: blue, country: green, mountain: brown, unclassified: grey)</td>
  </tr>
</table> 



We also made somse preliminary tests by assigning geographic coordinates found in our French wikiGazetteer to each node (headword). We have only 2535 nodes with coordinates over the 13734 nodes. 

Our first experiment is shown below. Colors identify clusters of nodes computed with the [modularity measure](https://en.wikipedia.org/wiki/Modularity_(networks)) implemented on Gephy.

<table><tr>
<td> <img src="img/geocodingEDDA1.png" width ="500"> </td>
<td> <img src="img/geocodingEDDA_network.png" width ="500" > </td>
</tr></table> 